In [121]:
# Audio to text

import os
import os
from tqdm import tqdm
import whisper
import subprocess
import string

os.environ["PATH"] += os.pathsep + r"C:\ffmpeg\bin"

def check_ffmpeg():
    try:
        cmd = ["ffmpeg", "-version"]
        result = subprocess.run(cmd, capture_output=True, text=True)
        if result.returncode == 0:
            print("ffmpeg is accessible")
        else:
            print("ffmpeg is not accessible")
    except FileNotFoundError:
        print("ffmpeg not found")

check_ffmpeg()

model = whisper.load_model("base")

def transcribe_audio(file_path):
    audio = whisper.load_audio(file_path)
    audio = whisper.pad_or_trim(audio)
    
    mel = whisper.log_mel_spectrogram(audio).to(model.device)
    
    _, probs = model.detect_language(mel)
    
    options = whisper.DecodingOptions(fp16=False)
    result = whisper.decode(model, mel, options)

    transcription = result.text.translate(str.maketrans('', '', string.punctuation)).lower()

    return transcription

flac_file_path = r'C:\CodeProjects\University\3.2_VU\deep learning\gmmGroup\audioResults\VitsModel\07282016HFUUforum_SLASH_07-28-2016_HFUUforum_DOT_mp3_00002.flac'

if os.path.isfile(flac_file_path):
    transcription = transcribe_audio(flac_file_path)
    print(f"Final Transcription: {transcription}")
else:
    print("File not found. Please check the file path.")


wav_directory = r'C:\CodeProjects\University\3.2_VU\deep learning\gmmGroup\audioResults\VitsModel'

transcriptions = []
for filename in tqdm(os.listdir(wav_directory), desc="Processing files"):
    if filename.endswith(".flac"):
        file_path = os.path.join(wav_directory, filename)
        transcription = transcribe_audio(file_path)
        transcriptions.append(transcription)



ffmpeg is accessible
Final Transcription: security so it doesnt feel very secure to me as a farmer to hear that in my family and i weve been farming here since 1990


Processing files: 100%|██████████| 335/335 [04:28<00:00,  1.25it/s]


In [122]:
# Save to txt

output_file_path = r'C:\CodeProjects\University\3.2_VU\deep learning\gmmGroup\audioResults\transcriptions_VitsModel.txt'
with open(output_file_path, 'w') as f:
    for transcription in transcriptions:
        f.write(transcription + '\n')

In [123]:
# txt to List

output_file_path = r'C:\CodeProjects\University\3.2_VU\deep learning\gmmGroup\audioResults\transcriptions_VitsModel.txt'
with open(output_file_path, 'r') as f:
    transcriptions_from_file = [line.strip() for line in f]

transcriptions_from_file

['i wanted this to share a few things but im going to not share as much as i wanted to share because we are starting late id like to get this thing going so we all get home at a decent hour this this election is very important to',
 'say it we support the greek culture to the tune of 04 no way i made a mistake this year they lowered it from 04 to 03 8 and in the same breath they are saying food',
 'security so it doesnt feel very secure to me as a farmer to hear that in my family and i weve been farming here since 1990',
 'three and we produce last year we produce 21000 pounds of food for the community on 2000 square feet so unless we were the deficient to produce that much food',
 'cummins gear in the spirit of being able to grow our food here locally so we started as an organization in actually in 2009 but really in 2000 and 10 march of 2000 and 10 we had our first meeting',
 'at ma and now we have ten chapters across the islands and about 900 members so we are growing exponentially 

In [124]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="clean.json")

textInput = [] 
audioPathInput = [] 

for data in dataset['train']['training_data']:
    textInput.extend(data['label'])
    audioPathInput.extend(data['name'])

printInputArrays = False

if printInputArrays:
    for i in range(len(textInput)):
        print(textInput[i])

    for i in range(len(audioPathInput)):
        print(audioPathInput[i])

In [125]:
# Calculate the WER between two texts

# Objective Evaluation
# Word Error Rate (WER):

# Transcribe the generated audio back to text using an automatic speech recognition (ASR) system.
# Compare the transcribed text with the original text to calculate the WER. The model with the lower WER is better.

from jiwer import wer

if len(textInput)==len(transcriptions_from_file):
    original_texts = textInput
    transcribed_text_model1 = transcriptions_from_file
    transcribed_text_model2 = []

    wer_list_model1 = [wer(original, transcribed) for original, transcribed in zip(original_texts, transcribed_text_model1)] if transcribed_text_model1 else []
    wer_list_model2 = [wer(original, transcribed) for original, transcribed in zip(original_texts, transcribed_text_model2)] if transcribed_text_model2 else []

    average_wer_model1 = sum(wer_list_model1) / len(wer_list_model1) if wer_list_model1 else float('inf')
    average_wer_model2 = sum(wer_list_model2) / len(wer_list_model2) if wer_list_model2 else float('inf')

    print(f"Average WER Model 1: {average_wer_model1}")
    print(f"Average WER Model 2: {average_wer_model2}")
else:
    print("Input arrays are not of the same length.")
    print(f"textInput: {len(textInput)} transcriptions_from_file: {len(transcriptions_from_file)}")


Average WER Model 1: 0.11969839372439199
Average WER Model 2: inf


In [126]:
# Perceptual Evaluation of Speech Quality (PESQ):

# Use PESQ to measure the quality of the speech signals. PESQ compares the generated audio to a reference audio signal and gives a quality score.
# Libraries like pypesq can be used for this purpose.

In [127]:
from pydub import AudioSegment
from pypesq import pesq
from scipy.io import wavfile
import os
from pypesq import pesq
from scipy.io import wavfile
import os

def convert_flac_to_wav(flac_file_path, wav_file_path):
    audio = AudioSegment.from_file(flac_file_path, format="flac")
    audio.export(wav_file_path, format="wav")

# File paths
ref_flac_path = r'C:\CodeProjects\University\3.2_VU\deep learning\gmmGroup\data\07282016HFUUforum_SLASH_07-28-2016_HFUUforum_DOT_mp3_00000.flac'
gen_flac_path_model1 = r'C:\CodeProjects\University\3.2_VU\deep learning\gmmGroup\audioResults\VitsModel\07282016HFUUforum_SLASH_07-28-2016_HFUUforum_DOT_mp3_00000.flac'
gen_flac_path_model2 = r'C:\CodeProjects\University\3.2_VU\deep learning\gmmGroup\audioResults\VitsModel\07282016HFUUforum_SLASH_07-28-2016_HFUUforum_DOT_mp3_00001.flac'


# File paths
ref_wav_path = r'C:\CodeProjects\University\3.2_VU\deep learning\gmmGroup\data\07282016HFUUforum_SLASH_07-28-2016_HFUUforum_DOT_mp3_00000.wav'
gen_wav_path_model1 = r'C:\CodeProjects\University\3.2_VU\deep learning\gmmGroup\audioResults\VitsModel\07282016HFUUforum_SLASH_07-28-2016_HFUUforum_DOT_mp3_00000.wav'
gen_wav_path_model2 = r'C:\CodeProjects\University\3.2_VU\deep learning\gmmGroup\audioResults\VitsModel\07282016HFUUforum_SLASH_07-28-2016_HFUUforum_DOT_mp3_00001.wav'


convert_flac_to_wav(ref_flac_path, ref_wav_path)
convert_flac_to_wav(gen_flac_path_model1, gen_wav_path_model1)
convert_flac_to_wav(gen_flac_path_model2, gen_wav_path_model2)

# Read WAV files
rate, ref_audio = wavfile.read(ref_wav_path)
rate, gen_audio_model1 = wavfile.read(gen_wav_path_model1)
rate, gen_audio_model2 = wavfile.read(gen_wav_path_model2)

# Compute PESQ scores
pesq_score_model1 = pesq(rate, ref_audio, gen_audio_model1, 'wb')
pesq_score_model2 = pesq(rate, ref_audio, gen_audio_model2, 'wb')

print(f"PESQ Score Model 1: {pesq_score_model1}")
print(f"PESQ Score Model 2: {pesq_score_model2}")

# Clean up temporary WAV files if needed
os.remove(ref_wav_path)
os.remove(gen_wav_path_model1)
os.remove(gen_wav_path_model2)


CouldntDecodeError: Decoding failed. ffmpeg returned error code: 3199971767

Output from ffmpeg/avlib:

ffmpeg version 2024-05-15-git-7b47099bc0-full_build-www.gyan.dev Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 13.2.0 (Rev5, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-librav1e --enable-libsvtav1 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --enable-libjxl --enable-libopenjpeg --enable-libvpx --en

In [ ]:
from pypesq import pesq
from scipy.io import wavfile

rate, ref_audio = wavfile.read(r'C:\CodeProjects\University\3.2_VU\deep learning\gmmGroup\data\07282016HFUUforum_SLASH_07-28-2016_HFUUforum_DOT_mp3_00000.flac')
rate, gen_audio_model1 = wavfile.read(r'C:\CodeProjects\University\3.2_VU\deep learning\gmmGroup\audioResults\VitsModel\07282016HFUUforum_SLASH_07-28-2016_HFUUforum_DOT_mp3_00000.flac')
rate, gen_audio_model2 = wavfile.read(r'C:\CodeProjects\University\3.2_VU\deep learning\gmmGroup\audioResults\VitsModel\07282016HFUUforum_SLASH_07-28-2016_HFUUforum_DOT_mp3_00001.flac')

pesq_score_model1 = pesq(rate, ref_audio, gen_audio_model1, 'wb')
pesq_score_model2 = pesq(rate, ref_audio, gen_audio_model2, 'wb')

print(f"PESQ Score Model 1: {pesq_score_model1}")
print(f"PESQ Score Model 2: {pesq_score_model2}")


ValueError: File format b'fLaC' not understood. Only 'RIFF' and 'RIFX' supported.